# OKGP epistasis: pull double variants from 1000 Genomes

Build a clean set of **double-variant (epistasis) IDs** from the 1000 Genomes Project (OKGP) for use in embedding and epistasis analyses.

**Pipeline:**
1. **Load or generate** 1000 Genomes chr12 double variants (pairs within a max distance, e.g. 100 bp).
   - Either load a pre-built CSV (`g1k_chr12_double_variants_max100bp.csv`) or run the VCF scan (optional, slow).
2. **Annotate** each pair with gene and strand (GENCODE) using the first variant's position.
3. **Build** `mut1`, `mut2`, and `epistasis_id` in the same format as other datasets (e.g. KRAS/FAS/MST1R).
4. **Save** to `okgp_epistasis.csv` (or `okgp_doubles_subset.csv`).

**Data sources:**
- 1000 Genomes high-coverage VCF (e.g. `20201028_CCDG_14151_B01_GRM_WGS_2020-08-05_chr12.recalibrated_variants.vcf.gz`).
- The same logic is used in `tcga_double_variants.ipynb` to produce `g1k_chr12_double_variants_max100bp.csv`; downstream `get_data.ipynb` and `running_okgp_doubles.ipynb` consume that CSV after adding gene/mut_id/epistasis_id.

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
tqdm.pandas()

# Ensure repo root is on path so "notebooks" can be imported
ROOT = Path.cwd()
for _ in range(4):
    if (ROOT / "notebooks" / "paper_data_config.py").exists():
        break
    ROOT = ROOT.parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

# Paths (set for your environment)
GENE_STRAND_PATH = "/tamir2/nicolaslynn/projects/dlm_wrappers/genebeddings/assets/benchmarks/gene_strands.csv"
GTF_PATH = "/tamir2/nicolaslynn/data/GENCODE/raw/gencode.v49.annotation.gtf.gz"

# Input: pre-built 1000G chr12 double variants (from tcga_double_variants.ipynb)
G1K_CHR12_CSV = "g1k_chr12_double_variants_max100bp.csv"
g1k_path = "/tamir2/nicolaslynn/projects/genomenet/genomenet/notebooks/data_generation_final/g1k_chr12_double_variants_max100bp.csv"

# Output under paper data root
from notebooks.paper_data_config import data_dir
OUT_OKGP_EPISTASIS = data_dir() / "okgp_epistasis.csv"

# Sampling: weight by theoretical distance distribution (n(d) = L - d for window length L)
SAMPLE_SIZE = 100_000   # total pairs to keep (or None to skip sampling)
L = 100                 # window length (match your double-variant window)
max_d = 100             # max distance (or your MAX_DISTANCE_BP)

def theoretical_weight(d):
    """Theoretical count for each d in "all pairs in window L": n(d) = L - d for d in 1..L-1."""
    if d < 1 or d >= L:
        return 0.0
    return float(L - d)

# Optional: filter by distance (bp). Set to None to keep all.
MAX_DISTANCE_BP = None  # e.g. 5 for adjacent only



## Shared helpers

In [ ]:
GENE_STRAND = pd.read_csv(GENE_STRAND_PATH).set_index("gene_name").Strand.to_dict()

def make_mut_id(row, pos_col="pos", ref_col="ref", var_col="alt", chrom_col="chrom", gene_col="gene", rev_col=None):
    if rev_col is None:
        rev = GENE_STRAND.get(row[gene_col], False)
    else:
        rev = row[rev_col]
    s = "N" if rev else "P"
    chrom = str(row[chrom_col]).strip("chr")
    return f"{row[gene_col]}:{chrom}:{int(row[pos_col])}:{row[ref_col]}:{row[var_col]}:{s}"

import pyranges as pr
genes_gr = pr.read_gtf(GTF_PATH)
genes = genes_gr[genes_gr.Feature == "gene"]

def annotate_gene_names(row):
    chrom = getattr(row, "chrom", row.get("Chromosome", "12"))
    chrom = str(chrom).strip("chr")
    if not chrom.startswith("chr"):
        chrom = f"chr{chrom}"
    pos = int(getattr(row, "pos1", row.get("pos1")))
    query = pr.PyRanges(chromosomes=[chrom], starts=[pos], ends=[pos + 1])
    hits = genes.join(query)
    if hits.df.empty:
        return pd.Series({"gene": np.nan, "rev": np.nan})
    df = hits.df
    df = df[df["gene_type"] == "protein_coding"]
    if df.empty:
        return pd.Series({"gene": np.nan, "rev": np.nan})
    is_plus = df["Strand"] == "+"
    df = df.copy()
    df["tss"] = np.where(is_plus, df["Start"], df["End"])
    df["dist_to_tss"] = (df["tss"] - pos).abs()
    best = df.loc[df["dist_to_tss"].idxmin()]
    rev = best["Strand"] == "-"
    return pd.Series({"gene": best["gene_name"], "rev": rev})

## 1. Load 1000 Genomes chr12 double variants

In [ ]:
# Resolve path: use g1k_path if file exists, else try G1K_CHR12_CSV relative paths
import os
if not os.path.isfile(g1k_path):
    for candidate in [G1K_CHR12_CSV, os.path.join("..", G1K_CHR12_CSV)]:
        if os.path.isfile(candidate):
            g1k_path = candidate
            break

okgp = pd.read_csv(g1k_path)
okgp = okgp.rename(columns={"distance_bp": "distance", "Chromosome": "chrom"})
if "chrom" not in okgp.columns:
    okgp["chrom"] = okgp.get("Chromosome", "chr12")
okgp["chrom"] = okgp["chrom"].astype(str).str.strip("chr")

if MAX_DISTANCE_BP is not None:
    okgp = okgp[okgp["distance"] <= MAX_DISTANCE_BP]

# Sample with theoretical distance distribution (L - d) so distance distribution matches "all pairs in window"
if SAMPLE_SIZE is not None and len(okgp) > SAMPLE_SIZE:
    okgp["weight"] = okgp["distance"].clip(lower=1).apply(theoretical_weight)
    okgp = okgp[okgp["weight"] > 0]
    weights = okgp["weight"] / okgp["weight"].sum()
    okgp = okgp.sample(n=SAMPLE_SIZE, weights=weights, random_state=42, replace=False).reset_index(drop=True)

print(f"Loaded {len(okgp)} pairs from {g1k_path}")
okgp.head()

## 2. Annotate gene and strand, build mut1 / mut2 / epistasis_id

In [ ]:
okgp[["gene", "rev"]] = okgp.progress_apply(annotate_gene_names, axis=1)
okgp["gene"] = okgp["gene"].fillna("GENE")

okgp["mut1"] = okgp.progress_apply(
    lambda r: make_mut_id(r, pos_col="pos1", ref_col="ref1", var_col="alt1", chrom_col="chrom", gene_col="gene", rev_col="rev"),
    axis=1,
)
okgp["mut2"] = okgp.progress_apply(
    lambda r: make_mut_id(r, pos_col="pos2", ref_col="ref2", var_col="alt2", chrom_col="chrom", gene_col="gene", rev_col="rev"),
    axis=1,
)
okgp["epistasis_id"] = okgp["mut1"] + "|" + okgp["mut2"]

print(okgp[["epistasis_id", "gene", "distance", "AF1", "AF2"]].head(10))

## 3. Save

In [ ]:
out_path = OUT_OKGP_EPISTASIS  # saved in current working directory
okgp.to_csv(out_path, index=False)
print(f"Saved {len(okgp)} rows to {out_path}")
okgp[["epistasis_id", "gene", "distance", "AF1", "AF2", "mut1", "mut2"]].head()

---
## Optional: Build g1k chr12 double variants from VCF

If you do not have `g1k_chr12_double_variants_max100bp.csv`, you can generate it from the 1000 Genomes VCF. Requires `pysam` and a local index (`.tbi`) for the chr12 VCF. This step is slow (~1–2 hours for chr12).

In [ ]:
# Uncomment and set paths to run.
# import pysam
#
# VCF_URL = (
#     "https://ftp.1000genomes.ebi.ac.uk/vol1/ftp/data_collections/"
#     "1000G_2504_high_coverage/working/20201028_3202_raw_GT_with_annot/"
#     "20201028_CCDG_14151_B01_GRM_WGS_2020-08-05_chr12.recalibrated_variants.vcf.gz"
# )
# LOCAL_TBI = "/path/to/20201028_CCDG_14151_B01_GRM_WGS_2020-08-05_chr12.recalibrated_variants.vcf.gz.tbi"
# CONTIG, MAX_DIST = "chr12", 100
#
# def is_biallelic_snv(rec):
#     if rec.alts is None or len(rec.alts) != 1: return False
#     return len(rec.ref) == 1 and len(rec.alts[0]) == 1
# def is_pass(rec):
#     f = set(rec.filter.keys())
#     return not f or "PASS" in f
#
# vcf = pysam.VariantFile(VCF_URL, index_filename=LOCAL_TBI)
# samples = list(vcf.header.samples)
# n_samples = len(samples)
# pairs, window = [], []
# for rec in tqdm(vcf.fetch(CONTIG), desc="Scanning " + CONTIG):
#     if not is_pass(rec) or (not is_biallelic_snv(rec)): continue
#     pos, ref, alt = rec.pos, rec.ref, rec.alts[0]
#     AF = rec.info.get("AF", np.nan)
#     if isinstance(AF, (list, tuple)): AF = AF[0] if AF else np.nan
#     carriers = np.array([any((a is not None) and (a >= 1) for a in rec.samples[s].get("GT") or ()) for s in samples], dtype=bool)
#     n_carriers = int(carriers.sum())
#     window = [w for w in window if pos - w["pos"] <= MAX_DIST]
#     for w in window:
#         dist = pos - w["pos"]
#         if dist <= 0 or dist > MAX_DIST: continue
#         both = int((carriers & w["carriers"]).sum())
#         pairs.append({"Chromosome": CONTIG, "pos1": w["pos"], "pos2": pos, "distance_bp": dist,
#                       "ref1": w["ref"], "alt1": w["alt"], "ref2": ref, "alt2": alt,
#                       "AF1": w["AF"], "AF2": AF, "n_carriers1": w["n_carriers"], "n_carriers2": n_carriers, "n_carriers_both": both,
#                       "pair_source": "g1k_chr12", "pair_type": "population_double_candidate"})
#     window.append({"pos": pos, "ref": ref, "alt": alt, "AF": AF, "carriers": carriers, "n_carriers": n_carriers})
# benign_pairs_chr12 = pd.DataFrame(pairs).sort_values(["pos1", "pos2"]).reset_index(drop=True)
# benign_pairs_chr12.to_csv("g1k_chr12_double_variants_max100bp.csv", index=False)
# print("Saved g1k_chr12_double_variants_max100bp.csv")


